In [6]:
!pip install gradio scikit-learn --quiet

In [22]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -----------------------------
# Global variables
# -----------------------------
inventory_df = None
items = []

# -----------------------------
# Load Excel
# -----------------------------
def load_excel(file):
    global inventory_df, items

    inventory_df = pd.read_excel(file.name)
    inventory_df["Date"] = pd.to_datetime(inventory_df["Date"])
    inventory_df.sort_values("Date", inplace=True)

    # Ensure Festival and State columns exist
    if "Festival" not in inventory_df.columns:
        inventory_df["Festival"] = None
    if "State" not in inventory_df.columns:
        inventory_df["State"] = None

    items = inventory_df["Item"].unique().tolist()

    return f"""
✅ Data Loaded Successfully!

📦 Items: {', '.join(items)}
📅 Records: {len(inventory_df)}
🌍 States: {inventory_df['State'].dropna().unique().tolist()}
🎉 Festivals: {inventory_df['Festival'].dropna().unique().tolist()}
"""

# -----------------------------
# LSTM Forecast Function
# -----------------------------
def lstm_forecast(item, days=7):
    df_item = inventory_df[inventory_df["Item"].str.lower() == item.lower()]

    if len(df_item) < 10:
        return None

    sales = df_item["Quantity_Sold"].values.reshape(-1, 1)
    scaler = MinMaxScaler()
    sales_scaled = scaler.fit_transform(sales)

    X, y = [], []
    window = 5

    for i in range(len(sales_scaled) - window):
        X.append(sales_scaled[i:i+window])
        y.append(sales_scaled[i+window])

    X, y = np.array(X), np.array(y)

    model = Sequential([
        LSTM(50, input_shape=(window, 1)),
        Dense(1)
    ])
    model.compile(optimizer="adam", loss="mse")
    model.fit(X, y, epochs=30, verbose=0)

    last_window = sales_scaled[-window:].reshape(1, window, 1)
    forecast = []

    for _ in range(days):
        pred = model.predict(last_window, verbose=0)[0][0]
        forecast.append(pred)
        last_window = np.append(last_window[:, 1:, :], [[[pred]]], axis=1)

    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    return max(0, int(np.sum(forecast)))

# -----------------------------
# Seasonal Trend Analysis
# -----------------------------
def seasonal_analysis(item, state=None):
    df_item = inventory_df[inventory_df["Item"].str.lower() == item.lower()]

    if state:
        df_item = df_item[df_item["State"].fillna("").str.lower() == state.lower()]

    if df_item.empty:
        return "⚠️ No data available for this item/state."

    festival_sales = df_item[df_item["Festival"].notna()]["Quantity_Sold"]
    normal_sales = df_item[df_item["Festival"].isna()]["Quantity_Sold"]

    if festival_sales.empty or normal_sales.empty:
        return "⚠️ Not enough seasonal data to calculate uplift."

    uplift = ((festival_sales.mean() - normal_sales.mean()) / normal_sales.mean()) * 100

    return f"""
📊 Seasonal Trend Analysis

Item: {item}
State: {state if state else 'All'}

Average Festival Sales: {festival_sales.mean():.1f}
Average Normal Sales: {normal_sales.mean():.1f}

📈 Demand Increase During Festivals: {uplift:.2f}%
"""

# -----------------------------
# Festival Uplift Adjustment
# -----------------------------
def festival_uplift(forecast, festival=None):
    uplift_map = {
        "diwali": 0.30,
        "pongal": 0.25,
        "onam": 0.20,
        "eid": 0.22,
        "navratri": 0.15
    }
    if festival:
        return int(forecast * (1 + uplift_map.get(festival.lower(), 0)))
    return forecast

# -----------------------------
# Recommendation Logic
# -----------------------------
def recommend(item, festival=None):
    forecast = lstm_forecast(item)
    if forecast is None:
        return "⚠️ Not enough data for forecasting."

    forecast = festival_uplift(forecast, festival)

    df_item = inventory_df[inventory_df["Item"].str.lower() == item.lower()]
    current_stock = int(df_item.iloc[-1]["Closing_Stock"])
    lead_time = int(df_item.iloc[-1]["Lead_Time"])

    safety_stock = 20
    required = forecast + safety_stock
    purchase_qty = max(0, required - current_stock)

    return f"""
📦 Inventory Forecast (Next 7 Days) - Inventory Batao

Item: {item}
Festival: {festival if festival else 'None'}

Current Stock: {current_stock}
Forecasted Demand: {forecast}
Recommended Purchase Quantity: {purchase_qty}

📌 Lead Time: {lead_time} days
"""

# -----------------------------
# Chatbot Function
# -----------------------------
def chatbot(query):
    query_lower = query.lower()

    for item in items:
        if item.lower() in query_lower:

            if "season" in query_lower or "festival" in query_lower:
                state = None
                for s in inventory_df["State"].dropna().unique():
                    if s.lower() in query_lower:
                        state = s
                        break
                return seasonal_analysis(item, state)

            festival = None
            for f in inventory_df["Festival"].dropna().unique():
                if f.lower() in query_lower:
                    festival = f
                    break

            return recommend(item, festival)

    return "❓ Please mention a valid item name."

# -----------------------------
# Gradio UI (INSTRUCTIONS PRESERVED)
# -----------------------------
with gr.Blocks(title="Inventory Batao") as app:
    gr.Markdown("""
# 🤖 Welcome to Inventory Batao
AI-Powered Inventory Forecasting with LSTM & Seasonal Trends

### 📄 Excel Format Instructions
Upload an Excel file with the following columns:

| Column Name         | Description                                    |
|--------------------|------------------------------------------------|
| Date                | YYYY-MM-DD                                     |
| Item                | Product name                                   |
| Opening_Stock       | Stock at start of day                          |
| Quantity_Purchased  | Purchased units                                |
| Quantity_Sold       | Sold units                                     |
| Closing_Stock       | Opening + Purchased − Sold                     |
| Lead_Time           | Supplier lead time (days)                      |
| State               | State / region (optional)                      |
| Festival            | Festival name (optional, leave empty if none) |

**Notes:**
- Festival column is optional.
- State column is optional.
- Closing stock will be used to calculate current inventory.
- You can ask questions like:
  - "How much milk should I purchase next week?"
  - "Festival demand analysis for rice in Tamil Nadu"
  - "Diwali impact on oil sales"
  - "Do I need to restock bread?"
""")

    file_input = gr.File(label="📊 Upload Your Inventory Excel File")
    load_output = gr.Textbox(label="File Status", lines=6)

    chatbot_input = gr.Textbox(label="Ask Inventory Batao")
    chatbot_output = gr.Textbox(label="Inventory Batao Response", lines=10)

    file_input.change(load_excel, inputs=file_input, outputs=load_output)
    chatbot_input.submit(chatbot, inputs=chatbot_input, outputs=chatbot_output)

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06facf2ba7dabf566a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
